<a href="https://colab.research.google.com/github/makeflower99/test/blob/master/Fastcamp_Pretreatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium
!pip install konlpy

In [11]:
from konlpy.tag import Okt
from collections import Counter
from konlpy.tag import Hannanum
import re
import pandas as pd
#크롤링한 결과 불러오기
# 'Fastcamp.csv'은 파일위치에서 최상위부분에 위치하여야한다.
#파일 넣는 방법 좌측에 폴더를 클릭한 후
# ..(상위폴더로 이동)을 선택한 후 파일 업로드 버튼을 눌러 업로드 한다.
# 저장된 파일은 content폴더에 위치해있습니다.


#data 불러오기
df = pd.read_csv("/Fastcamp.csv")

#한글기준으로 문자 전부 추출
hannanum = Hannanum()
nouns = []
for i in df["강좌명"]:
  nouns.extend(hannanum.nouns(i))

df_1 = pd.DataFrame({"명사": nouns})#한글로 된 아이들만 빼서 df_1
df['영어강좌명'] = ''

# "강좌명" 열의 각 행에 대해 처리
for i in range(len(df)):
    course_name = df.loc[i, '강좌명']

    # 영어만 추출하여 새로운 열에 저장
    english_part = re.sub('[^a-zA-Z\s]', '', course_name)
    df.at[i, '영어강좌명'] = english_part

english_words = []

# "영어강좌명" 열의 값이 있는 경우, 띄어쓰기 단위로 단어 추출하여 리스트에 추가
for english_course in df['영어강좌명']:
    if pd.notna(english_course):
        words = english_course.split()
        english_words.extend(words)
english_words_df = pd.DataFrame(english_words, columns=["영어단어추출"])

result_df = pd.concat([df_1, english_words_df], axis=1)
result_df = result_df[~result_df["명사"].isin(["&", "1", "2", "3"])]

#한글중에 가장 많이 나온 단어
korean_rank=pd.DataFrame(result_df["명사"].value_counts())
english_rank=pd.DataFrame(result_df["영어단어추출"].value_counts())

korean_rank.to_csv("한글순위.csv",index=False)
english_rank.to_csv("영어순위.csv",index=False)

